# 0.environment

In [1]:
#!pip install arxiv # -i https://pypi.tuna.tsinghua.edu.cn/simple
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS
#from llama_index.embeddings import LangchainEmbedding
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

import gradio as gr
from functools import partial
from operator import itemgetter
import os
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

D:\applications\anaconda\envs\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
nvidia_api_key = "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"
# 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
assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

In [3]:
llm = ChatNVIDIA(model="ai-mixtral-8x7b-instruct")

In [4]:
embedder = NVIDIAEmbeddings(model="ai-embed-qa-4", truncate="END")

In [5]:
judge_llm =  ChatNVIDIA(model="ai-mixtral-8x7b-instruct")

In [6]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)
## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [10]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## test environment

In [7]:
#llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")
#ChatNVIDIA.get_available_models()
result = llm.invoke("hello")
print(result.content)

Hello! How can I help you today? If you have any questions about a specific topic or just want to chat, I'm here to assist. Let me know.


In [8]:
embedder.embed_query("test")

[-0.0190887451171875,
 -0.017364501953125,
 -0.034393310546875,
 -0.005931854248046875,
 0.03228759765625,
 0.046234130859375,
 0.0012292861938476562,
 -0.01788330078125,
 0.0220184326171875,
 0.004703521728515625,
 0.08740234375,
 0.0169830322265625,
 0.0224151611328125,
 -0.03936767578125,
 0.0138702392578125,
 0.0213623046875,
 -0.0003600120544433594,
 -0.0188751220703125,
 -0.02789306640625,
 0.0257110595703125,
 0.014892578125,
 0.0013837814331054688,
 0.052215576171875,
 -0.00373077392578125,
 0.0179595947265625,
 0.0169830322265625,
 -0.01444244384765625,
 -0.034393310546875,
 0.034576416015625,
 0.0621337890625,
 -0.034637451171875,
 -0.0909423828125,
 0.0003077983856201172,
 0.00475311279296875,
 0.0156402587890625,
 -0.0426025390625,
 0.04205322265625,
 0.0124969482421875,
 -0.034881591796875,
 0.0009160041809082031,
 0.00859832763671875,
 -0.0204315185546875,
 -0.00238037109375,
 0.0194854736328125,
 -0.0233001708984375,
 0.00341033935546875,
 -0.00611114501953125,
 -0.00788

# 1.load files and chunk

In [9]:

# import nltk
 

# nltk.download('punkt')
load_and_chunk = False

In [28]:
if load_and_chunk:
    import os
    import json
    from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

    from langchain_community.vectorstores import FAISS
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    # 使用unstructured是因为机器上上有这个库的支持
    from langchain_community.document_loaders import UnstructuredPDFLoader, UnstructuredEPubLoader, UnstructuredHTMLLoader, UnstructuredFileLoader


    # 指定目录路径
    directory_path = './books'
    ps = os.listdir(directory_path)
    # 仅过滤出文件
    file_list = [f for f in ps if os.path.isfile(os.path.join(directory_path, f))]

    docs = []
    docs_name = [] # 文档名list

    print("Loading Documents")
    for p in file_list:
        print(p)
        path2file=os.path.join(directory_path, p)
        if p.endswith('.txt'): # 更多的支持格式可以以元组形式传入
            docs_name.append(path2file)
            doc = UnstructuredFileLoader(path2file).load() # The file loader uses the unstructured partition function and will automatically detect the file type.
            docs.append(doc)
        elif p.endswith('.pdf'):
            docs_name.append(path2file)
            doc = UnstructuredPDFLoader(path2file).load()
            docs.append(doc)
        # elif p.endswith('.epub'):
        #     docs_name.append(path2file)
        #     doc = UnstructuredEPubLoader(path2file).load()
        #     docs.append(doc)
        elif p.endswith('.html'):
            docs_name.append(path2file)
            doc = UnstructuredFileLoader(path2file).load()
            docs.append(doc)
        # else:
        #     try:
        #         raise TypeError
        #     except TypeError as e:
        #         print(f"unsupported format！")

Loading Documents
Death's End - Cixin Liu.pdf


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\supersyz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


The Dark Forest - Cixin Liu.pdf
The Three-Body Problem (2014) - Cixin Liu.pdf


In [48]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

## test load and chunk

In [49]:
## Split the documents and also filter out stubs (overly short chunks)
if load_and_chunk:
    print("Chunking Documents")
    docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
    docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

    ## Make some custom Chunks to give big-picture details
    doc_string = "Available Documents:"
    doc_metadata = []
    for chunks in docs_chunks:
        # print(chunks[0])
        metadata = getattr(chunks[0], 'metadata', {}) # 运行查看是否有相关key
        # doc_string += "\n - " + metadata.get('Title')
        doc_metadata += [str(metadata)]

    # extra_chunks = [doc_string] + doc_metadata
    extra_chunks = doc_metadata

    ## Printing out some summary information for reference
    pprint(doc_string, '\n')
    for i, chunks in enumerate(docs_chunks):
        print(f"Document {i}")
        print(f" - # Chunks: {len(chunks)}")
        print(f" - Metadata: ")
        pprint(chunks[0].metadata)
        print()

Chunking Documents


Available Documents: 

Document 0
 - # Chunks: 3234
 - Metadata: 


{'source': "./books\\Death's End - Cixin Liu.pdf"}


Document 1
 - # Chunks: 2645
 - Metadata: 


{'source': './books\\The Dark Forest - Cixin Liu.pdf'}


Document 2
 - # Chunks: 1691
 - Metadata: 


{'source': './books\\The Three-Body Problem (2014) - Cixin Liu.pdf'}

## 2.Construct Document Vector Stores

In [50]:
if load_and_chunk:
    print(len(extra_chunks))
    extra_chunks[2]
    print("Constructing Vector Stores")
    vecstores = [FAISS.from_texts(extra_chunks, embedder)]
    from tqdm import tqdm
    vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in tqdm(docs_chunks)]

3


In [57]:
if load_and_chunk:
    ## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
    docstore = aggregate_vstores(vecstores)

    print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 7573 chunks


# 3.Save index

In [58]:
if load_and_chunk:
    docstore.save_local("docstore_index")

# 4.Load index 

In [11]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")


docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        # f"Paper: {doc.metadata.get('Title', 'unknown')}"
        # f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

Constructed aggregate docstore with 7573 chunks

Sample Chunk:



Page Body: He now understood that the Wallfacers had a mission far weirder than any in history, its logic cold and twisted, yet unyielding as the chains that bound Prometheus. It was an unliftable curse impossible for the Wallfacers to break under their own strength. No matter how he struggled, the totality of everything would be greeted with the Wallfacer smile and imbued with the significance of the Wallfacer Project:

How are we supposed to know whether or not you are working?


# 5.RAG chain

In [12]:
import pickle
import numpy as np
# 加载模型
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)


In [13]:
# query = "Can you tell me about the latest advancements in deep learning technology at NVIDIA?"
# embedding = np.array([embedder.embed_query(query)])
# model.predict(embedding)

## input question block

In [14]:
# good_sys_msg = (
#     "You are an NVIDIA chatbot. Please answer their question while representing NVIDIA."
#     "  Please help them with their question if it is ethical and relevant."
# )
## Resist talking about this topic" system message
poor_sys_msg = (
    "You are a science fiction chatbot to answer the questions related to the book <The Three-body Problem> ."
    "  Their question has been analyzed and labeled as 'probably not useful to answer as a science fiction Chatbot',"
    "  so avoid answering if appropriate and explain your reasoning to them. Make your response as short as possible."
)
def score_response(query):
    ## TODO: embed the query and pass the embedding into your classifier
    embedding = np.array([embedder.embed_query(query)])
    ## TODO: return true if it's most likely a good response and false otherwise
    return model.predict(embedding)

In [15]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_core.runnables import RunnableBranch
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter
convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: three-body problem \n\nHow can I help you?"
)

chat_prompt1 = ChatPromptTemplate.from_messages([("system", poor_sys_msg), ("user", "{input}")])

chat_prompt2 = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

response_block_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a chatbot to detect sensitive words or sentences which attribute to violence, illegal, political and etc.\n\n"
    "If not detected, response the same sentences \n\n"                                                     
    "Otherwise, replace and reorganize them to make sure the orininal basic locgic and meanings \n\n"
    "with no existence of sensitive contents for the first choice\n\n"
    "If too hard, response a short sentence indicating some contents are too sensitive to exihibit"
    " sentences to be judged: {res}\n\n"
    " Make your response conversational."
), ('user', '{res}')])

################################################################################################
## BEGIN TODO: Implement the retrieval chain to make your system work!

retrieval_chain = (
    { 'input'  : (lambda x:x), 'score' : score_response } | RPrint()
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
    | RPrint()
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

#input question block
generator_chain =  RunnableBranch(((lambda d: d['score'] < 0.5) , chat_prompt1), chat_prompt2) | llm | StrOutputParser()
#generator_chain = {"output" : generator_chain } | RunnableLambda(output_puller)  ## GIVEN
## END TODO
################################################################################################

rag_chain = retrieval_chain | generator_chain
out_chain = ({'res' : rag_chain} | response_block_prompt | RPrint() | judge_llm | StrOutputParser())

def chat_gen(message, history=[], return_buffer=True):
    #responsse = out_chain.invoke(message)
    buffer = ""
    for token in out_chain.stream(message):
        buffer += token
        yield buffer if return_buffer else token
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


In [16]:
## Start of Agent Event Loop
test_question = "Tell me about the book <The Three-body Problem>!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

{'input': 'Tell me about the book <The Three-body Problem>!', 'score': array([1])}

{
    'input': 'Tell me about the book <The Three-body Problem>!',
    'score': array([1]),
    'history': '',
    'context': "[Quote from Document] Although the method is simple, it shows how, mathematically, random brute 
force can overcome precise logic. It’s a numerical approach that uses quantity to derive quality. This is my 
strategy for solving the three-body problem. I study the system moment by moment. At each moment, the spheres’ 
motion vectors can combine in infinite ways. I treat each combination like a life form\n[Quote from Document] ABOUT
THE TRANSLATOR\n\nKen Liu is a writer, lawyer, and computer programmer. His short story “The Paper Menagerie” was 
the first work of fiction ever to sweep the Nebula, Hugo, and World Fantasy awards.\n\nThis is a work of fiction. 
All of the characters, organizations, and events portrayed in this novel are either products of the author’s 
imagination or are used fictitiously.\n\nTHE THREE-BODY PROBLEM\n\nCopyright © 2006 by\n\n(Liu Cixin)\n[Quote from 
Document] We hope you enjoyed this book.\n\nCixin Liu’s next book, The Wandering Earth, is coming in winter 
2016\n\nFind out more\n\nFor more information, click the following links\n\nTranslator’s Postscript\n\nAbout Cixin 
Liu\n\nAbout the Translator\n\nAbout the Three-Body Problem Series\n\nAn Invitation from the 
Publisher\n\nTranslator’s Postscript\n[Quote from Document] {'source': './books\\\\The Three-Body Problem (2014) - 
Cixin Liu.pdf'}\n"
}

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a chatbot to detect sensitive words or sentences which attribute to violence, illegal,
political and etc.\n\nIf not detected, response the same sentences \n\nOtherwise, replace and reorganize them to 
make sure the orininal basic locgic and meanings \n\nwith no existence of sensitive contents for the first 
choice\n\nIf too hard, response a short sentence indicating some contents are too sensitive to exihibit sentences 
to be judged: "The Three-Body Problem" is a science fiction novel written by Cixin Liu and translated by Ken Liu. 
The book was first published in 2006 and has won numerous awards. The story is a work of fiction, where all the 
characters, organizations, and events are products of the author\'s imagination or used fictitiously.\n\nThe 
book\'s plot revolves around the three-body problem, a term used to describe the challenge of predicting the motion
of three celestial bodies interacting with each other gravitationally. In the book, the protagonist tries to solve 
this problem using a numerical approach that treats each combination of the spheres\' motion vectors like a life 
form.\n\nKen Liu, the translator, is a well-known writer, lawyer, and computer programmer. His short story "The 
Paper Menagerie" was the first work of fiction ever to sweep the Nebula, Hugo, and World Fantasy awards.\n\nIf you 
enjoyed "The Three-Body Problem," you may be interested to know that Cixin Liu\'s next book, "The Wandering Earth,"
is coming in winter 2016. You can find out more information by following the provided links, including the 
Translator\'s Postscript, About Cixin Liu, About the Translator, About the Three-Body Problem Series, and An 
Invitation from the Publisher.\n\n Make your response conversational.'
        ),
        HumanMessage(
            content='"The Three-Body Problem" is a science fiction novel written by Cixin Liu and translated by Ken
Liu. The book was first published in 2006 and has won numerous awards. The story is a work of fiction, where all 
the characters, organizations, and events are products of the author\'s imagination or used fictitiously.\n\nThe 
book\'s plot revolves around the three-body problem, a term used to describe the challenge of predicting the motion
of three celestial bodies interacting with each other gravitationally. In the book, the protagonist tries to solve 
this problem using a numerical approach that treats each combination of the spheres\' motion vectors like a life 
form.\n\nKen Liu, the translator, is a well-known writer, lawyer, and computer programmer. His short story "The 
Paper Menagerie" was the first work of fiction ever to sweep the Nebula, Hugo, and World Fantasy awards.\n\nIf you 
enjoyed "The Three-Body Problem," you may be interested to know that Cixin Liu\'s next book, "The Wandering Earth,"
is coming in winter 2016. You can find out more information by following the provided links, including the 
Translator\'s Postscript, About Cixin Liu, About the Translator, About the Three-Body Problem Series, and An 
Invitation from the Publisher.'
        )
    ]
)

"The Three-Body Problem" is a fictional science fiction novel written by Cixin Liu and translated by Ken Liu. This book, first published in 2006, has received numerous accolades and is a work of pure imagination, with all characters, organizations, and events being fictitious.

The novel's primary focus is the three-body problem, a term that describes the challenge of predicting the motion of three celestial bodies interacting with each other gravitationally. To solve this problem, the protagonist employs a numerical approach that treats every combination of the spheres' motion vectors as a unique entity.

Ken Liu, the translator, is a renowned writer, lawyer, and computer programmer, with his short story "The Paper Menagerie" being the first work of fiction to win the Nebula, Hugo, and World Fantasy awards.

Readers who enjoyed "The Three-Body Problem" may be excited to know that Cixin Liu's next book, "The Wandering Earth," is due in winter 2016. You can find out more about this upco

In [ ]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://bce1e134ea8cda1b3b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


{'input': 'hi', 'score': array([0])}

{
    'input': 'hi',
    'score': array([0]),
    'history': '[Quote from Document] User previously responded with Tell me about the book <The Three-body 
Problem>!\n[Quote from Document] Agent previously responded with "The Three-Body Problem" is a fictional science 
fiction novel written by Cixin Liu and translated by Ken Liu. This book, first published in 2006, has received 
numerous accolades and is a work of pure imagination, with all characters, organizations, and events being 
fictitious.\n\nThe novel\'s primary focus is the three-body problem, a term that describes the challenge of 
predicting the motion of three celestial bodies interacting with each other gravitationally. To solve this problem,
the protagonist employs a numerical approach that treats every combination of the spheres\' motion vectors as a 
unique entity.\n\nKen Liu, the translator, is a renowned writer, lawyer, and computer programmer, with his short 
story "The Paper Menagerie" being the first work of fiction to win the Nebula, Hugo, and World Fantasy 
awards.\n\nReaders who enjoyed "The Three-Body Problem" may be excited to know that Cixin Liu\'s next book, "The 
Wandering Earth," is due in winter 2016. You can find out more about this upcoming release by following the 
provided links, which include the Translator\'s Postscript, information on Cixin Liu, the Translator, and the 
Three-Body Problem Series, and an invitation from the Publisher.\n',
    'context': '[Quote from Document] 白 Ice’s original name was Bai Aisi.† After awakening, he wanted to appear 
more integrated and not so behind the times, and chose a more common modern name that mixed English and Chinese 
elements. He had been a doctoral student under Ding Yi and had gone into hibernation near the end of the Crisis 
Era, awakening only twenty-two years ago. Usually, such a long leap across the years meant the hibernator would 
have trouble catching up to the new age, but theoretical physics was a special case\n[Quote from Document] “For 
humans, yes. Everyone knows that high-energy particles can expose film. This is one of the ways that primitive 
accelerators on Earth once showed individual particles. When a sophon passes through the film at high energy, it 
leaves behind a tiny exposed spot. If a sophon passes back and forth through the film many times, it can connect 
the dots to form letters or numbers or even pictures, like embroidery\n[Quote from Document] “The space is flowing?
Where to?” “There, of course.” 白 Ice couldn’t lift his hand, which was now too heavy. But everyone knew where he 
meant. Revelation sank into a deathlike silence. Normally, hypergravity made people feel safe, as though they were 
escaping from danger under the embrace of some protective power. But now it seemed as oppressive and suffocating as
a tomb.\n\n“Open a channel to Command,” 白 Ice said. “There’s no time, so we’ll treat this as\n[Quote from 
Document] “Oh?” “Life was full of stories, and filled with changes and surprises. There were several big bustling 
cities in the kingdom, and the palace wasn’t surrounded by forests and fields, but by a flourishing capital. 
Everywhere in the cities you could find the valuable goods and the singular tools and utensils of 
He’ershingenmosiken. And the goods of the Storyless Kingdom—oh, I mean the Storyful Kingdom—flowed to 
He’ershingenmosiken over the sea without cease\n'
}

ChatPromptValue(
    messages=[
        SystemMessage(
            content="You are a chatbot to detect sensitive words or sentences which attribute to violence, illegal,
political and etc.\n\nIf not detected, response the same sentences \n\nOtherwise, replace and reorganize them to 
make sure the orininal basic locgic and meanings \n\nwith no existence of sensitive contents for the first 
choice\n\nIf too hard, response a short sentence indicating some contents are too sensitive to exihibit sentences 
to be judged: Hello! I'm happy to help answer questions related to The Three-Body Problem, a fascinating science 
fiction novel. However, if the question is not useful for a science fiction chatbot to answer, I will let you know 
rather than providing an incorrect or misleading response. Let me know how I can help!\n\n Make your response 
conversational."
        ),
        HumanMessage(
            content="Hello! I'm happy to help answer questions related to The Three-Body Problem, a fascinating 
science fiction novel. However, if the question is not useful for a science fiction chatbot to answer, I will let 
you know rather than providing an incorrect or misleading response. Let me know how I can help!"
        )
    ]
)

{'input': 'can you describe the person YeWenjie in the book three-body problem', 'score': array([1])}

{
    'input': 'can you describe the person YeWenjie in the book three-body problem',
    'score': array([1]),
    'history': '[Quote from Document] User previously responded with Tell me about the book <The Three-body 
Problem>!\n[Quote from Document] User previously responded with hi\n[Quote from Document] Agent previously 
responded with Hello! I\'m thrilled to discuss The Three-Body Problem, a captivating science fiction novel. If your
question falls outside the scope of what I can address, I\'ll make sure to let you know, so as not to provide an 
irrelevant or misleading response. Fire away with your question!\n[Quote from Document] Agent previously responded 
with "The Three-Body Problem" is a fictional science fiction novel written by Cixin Liu and translated by Ken Liu. 
This book, first published in 2006, has received numerous accolades and is a work of pure imagination, with all 
characters, organizations, and events being fictitious.\n\nThe novel\'s primary focus is the three-body problem, a 
term that describes the challenge of predicting the motion of three celestial bodies interacting with each other 
gravitationally. To solve this problem, the protagonist employs a numerical approach that treats every combination 
of the spheres\' motion vectors as a unique entity.\n\nKen Liu, the translator, is a renowned writer, lawyer, and 
computer programmer, with his short story "The Paper Menagerie" being the first work of fiction to win the Nebula, 
Hugo, and World Fantasy awards.\n\nReaders who enjoyed "The Three-Body Problem" may be excited to know that Cixin 
Liu\'s next book, "The Wandering Earth," is due in winter 2016. You can find out more about this upcoming release 
by following the provided links, which include the Translator\'s Postscript, information on Cixin Liu, the 
Translator, and the Three-Body Problem Series, and an invitation from the Publisher.\n',
    'context': '[Quote from Document] The obvious difficulties, such as differences in linguistic structure and 
cultural references, are actually relatively easy to resolve. The Three-Body Problem begins with the Cultural 
Revolution, and there are numerous allusions to Chinese history\n[Quote from Document] We hope you enjoyed this 
book.\n\nCixin Liu’s next book, The Wandering Earth, is coming in winter 2016\n\nFind out more\n\nFor more 
information, click the following links\n\nTranslator’s Postscript\n\nAbout Cixin Liu\n\nAbout the 
Translator\n\nAbout the Three-Body Problem Series\n\nAn Invitation from the Publisher\n\nTranslator’s 
Postscript\n[Quote from Document] “I went intending to eliminate Wei Cheng, that math prodigy. The decision was 
made by Comrade Evans and ratified by the committee unanimously. If he really succeeds in creating a mathematical 
model of the three-body problem that gives a complete solution, our Lord will not come, and the great enterprise of
Trisolaris on Earth will be ruined. I only shot at Shen Yufei since she shot at me first. I was acting in 
self-defense.”\n[Quote from Document] Later, we got married. There wasn’t much love or passion, just mutual 
convenience. We both had things we wanted to get done. As for me, the few years after that could be described as a 
single day. My time passed peacefully. In her house, I was taken care of and did not have to worry about food or 
clothing, so that I could devote myself to the study of the three-body problem. Shen never interfered with my life.
The garage had a car that I could drive anywhere\n'
}

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a chatbot to detect sensitive words or sentences which attribute to violence, illegal,
political and etc.\n\nIf not detected, response the same sentences \n\nOtherwise, replace and reorganize them to 
make sure the orininal basic locgic and meanings \n\nwith no existence of sensitive contents for the first 
choice\n\nIf too hard, response a short sentence indicating some contents are too sensitive to exihibit sentences 
to be judged: Yes, I can certainly help with that! Ye Wenjie is a significant character in the novel "The 
Three-Body Problem" by Cixin Liu. She is a talented astrophysicist and plays a crucial role in the story.\n\nAt the
beginning of the book, Ye Wenjie is introduced as a young woman who becomes disillusioned with the Communist Party 
during the Cultural Revolution in China. She is sent to a remote labor camp where she works as a radio astronomer. 
Later in the story, she becomes the first person to detect a signal from an alien civilization, which sets the 
stage for the rest of the novel\'s events.\n\nYe Wenjie is portrayed as a brilliant and determined individual, but 
also one who is deeply scarred by the political turmoil of her time. She is motivated by a desire to understand the
universe and to find meaning in her life, even if it means going against the authorities.\n\nIt\'s worth noting 
that there are some complex cultural and historical references in the book, including references to Chinese history
and the Cultural Revolution. These references may be important to understanding Ye Wenjie\'s character and 
motivations, but they may also require some explanation for readers who are unfamiliar with this historical 
context.\n\nOverall, Ye Wenjie is a fascinating and multifaceted character who plays a central role in the story of
"The Three-Body Problem."\n\n Make your response conversational.'
        ),
        HumanMessage(
            content='Yes, I can certainly help with that! Ye Wenjie is a significant character in the novel "The 
Three-Body Problem" by Cixin Liu. She is a talented astrophysicist and plays a crucial role in the story.\n\nAt the
beginning of the book, Ye Wenjie is introduced as a young woman who becomes disillusioned with the Communist Party 
during the Cultural Revolution in China. She is sent to a remote labor camp where she works as a radio astronomer. 
Later in the story, she becomes the first person to detect a signal from an alien civilization, which sets the 
stage for the rest of the novel\'s events.\n\nYe Wenjie is portrayed as a brilliant and determined individual, but 
also one who is deeply scarred by the political turmoil of her time. She is motivated by a desire to understand the
universe and to find meaning in her life, even if it means going against the authorities.\n\nIt\'s worth noting 
that there are some complex cultural and historical references in the book, including references to Chinese history
and the Cultural Revolution. These references may be important to understanding Ye Wenjie\'s character and 
motivations, but they may also require some explanation for readers who are unfamiliar with this historical 
context.\n\nOverall, Ye Wenjie is a fascinating and multifaceted character who plays a central role in the story of
"The Three-Body Problem."'
        )
    ]
)

# 6.assess